In [1]:
"""
Example demonstrating:
1. Pydantic schemas for US addresses
2. Structured output with LLM calls
3. Converting responses to Python objects
4. Generalizing structured output to tool use

Sections:
    ### Schemas
    ### Structured Output
    ### Converting the response
    ### Converting `tool_use` to Python objects
    ## Generalizing Structure Output to Tool Use
"""


from enum import Enum

from annotated_types import Annotated, Optional, Predicate

In [2]:
### Schemas
from pydantic import BaseModel


class USStateEnum(str, Enum):
    """
    An enumeration of US state and territory abbreviations.
    """

    AK = "AK"
    AL = "AL"
    AR = "AR"
    AZ = "AZ"
    CA = "CA"
    CO = "CO"
    CT = "CT"
    DE = "DE"
    FL = "FL"
    GA = "GA"
    HI = "HI"
    IA = "IA"
    ID = "ID"
    IL = "IL"
    IN = "IN"
    KS = "KS"
    KY = "KY"
    LA = "LA"
    MA = "MA"
    MD = "MD"
    ME = "ME"
    MI = "MI"
    MN = "MN"
    MO = "MO"
    MS = "MS"
    MT = "MT"
    NC = "NC"
    ND = "ND"
    NE = "NE"
    NH = "NH"
    NJ = "NJ"
    NM = "NM"
    NV = "NV"
    NY = "NY"
    OH = "OH"
    OK = "OK"
    OR = "OR"
    PA = "PA"
    RI = "RI"
    SC = "SC"
    SD = "SD"
    TN = "TN"
    TX = "TX"
    UT = "UT"
    VA = "VA"
    VT = "VT"
    WA = "WA"
    WI = "WI"
    WV = "WV"
    WY = "WY"
    DC = "DC"
    AS = "AS"
    GU = "GU"
    MP = "MP"
    PR = "PR"
    VI = "VI"


class USPostalAddress(BaseModel):
    """
    A postal address model restricted to U.S. states and territories.
    """

    name: Annotated[Optional[str], "The name of the person or organization at this address"]
    street: Annotated[str, "The street address of the location"]
    city: Annotated[str, "The city of the location"]
    state: Annotated[USStateEnum, "The state of the location expressed as two uppercase letters"]
    zip: Annotated[
        str,
        Predicate(lambda x: len(x) == 5),
        Predicate(lambda x: all([i.isdigit() for i in x])),
        "The zip code of the location, expressed as a five digit number",
    ]


# Example usage of the schema
address = USPostalAddress(
    name="Christopher Brooks",
    street="105 North State Street",
    city="Ann Arbor",
    state="MI",
    zip="48109",
)
print(address.json())

{"name":"Christopher Brooks","street":"105 North State Street","city":"Ann Arbor","state":"MI","zip":"48109"}


In [3]:
### Structured Output
from langchain_aws.chat_models import ChatBedrockConverse
from langchain_core.prompts.chat import ChatPromptTemplate

task = "What is the address of the School of Information at the University of Michigan?"
prompt = ChatPromptTemplate([("human", task)])

llm_llama3_70b_instruct = ChatBedrockConverse(
    model="us.meta.llama3-1-70b-instruct-v1:0",
    region_name="us-east-1",
)

# Convert the base LLM into one that outputs structured data of type USPostalAddress
structured_llm = llm_llama3_70b_instruct.with_structured_output(USPostalAddress)
chain = prompt | structured_llm

response = chain.invoke({})
print(response)
print(type(response))

# Compare the original model and the new structured model
print(type(llm_llama3_70b_instruct))
print(type(structured_llm))

# The structured model is actually a RunnableSequence, so we can see what steps it contains
for step in structured_llm.steps:
    print(type(step))

# The first step is the bound attribute of the RunnableBinding object
first_step = structured_llm.steps[0]
print(type(first_step.bound))

print("Original model")
# to_json() returns a dict representation of the model’s state
for key, value in llm_llama3_70b_instruct.to_json().items():
    print(f"\t{key}: {value}")

print("\nStructured model")
for key, value in first_step.to_json().items():
    print(f"\t{key}: {value}")

# Inspect the kwargs of the first step, which contains the bound model and output type
print("\nInspecting 'kwargs' in the structured model step:")
for key, value in first_step.to_json()["kwargs"].items():
    print(f"{key}: {value}")

None
<class 'NoneType'>
<class 'langchain_aws.chat_models.bedrock_converse.ChatBedrockConverse'>
<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.runnables.base.RunnableBinding'>
<class 'langchain_aws.function_calling.ToolsOutputParser'>
<class 'langchain_aws.chat_models.bedrock_converse.ChatBedrockConverse'>
Original model
	lc: 1
	type: constructor
	id: ['langchain_aws', 'chat_models', 'ChatBedrockConverse']
	kwargs: {'disable_streaming': 'tool_calling', 'model_id': 'us.meta.llama3-1-70b-instruct-v1:0', 'region_name': 'us-east-1', 'provider': 'us', 'supports_tool_choice_values': ()}
	name: ChatBedrockConverse

Structured model
	lc: 1
	type: constructor
	id: ['langchain', 'schema', 'runnable', 'RunnableBinding']
	kwargs: {'bound': ChatBedrockConverse(disable_streaming='tool_calling', client=<botocore.client.BedrockRuntime object at 0x77798e2037f0>, model_id='us.meta.llama3-1-70b-instruct-v1:0', region_name='us-east-1', provider='us', supports_tool_choice_

In [4]:
### Converting the response
# Invoking first_step with a question to see how the model responds
first_step.invoke("What is the address of the School of Information at the University of Michigan?")

no_tool = None
used_tool = None
while no_tool is None or used_tool is None:
    response = first_step.invoke("What is the address of the School of Information at the University of Michigan?")
    # The response is an AIMessage with tool_calls we can examine
    for tpl in response:
        if tpl[0] == "tool_calls":
            if len(tpl[1]) == 0:
                no_tool = response
            else:
                used_tool = response

print("Response without tool usage")
print(f"content={no_tool.content}")
print(no_tool.tool_calls)

print("----")
print("Response with tool usage")
print(f"content={used_tool.content}")
print(used_tool.tool_calls)

Response without tool usage
content=

The address of the School of Information at the University of Michigan is:

School of Information
University of Michigan
105 S State St
Ann Arbor, MI 48109-1285
[]
----
Response with tool usage
content=[{'type': 'tool_use', 'name': 'USPostalAddress', 'input': {'name': 'School of Information, University of Michigan', 'street': '105 S State St', 'city': 'Ann Arbor', 'state': 'MI', 'zip': '48109'}, 'id': 'tooluse_Lv8_yGV0SLOAeJYSCHuEjQ'}]
[{'name': 'USPostalAddress', 'args': {'name': 'School of Information, University of Michigan', 'street': '105 S State St', 'city': 'Ann Arbor', 'state': 'MI', 'zip': '48109'}, 'id': 'tooluse_Lv8_yGV0SLOAeJYSCHuEjQ', 'type': 'tool_call'}]


In [5]:
### Converting `tool_use` to Python objects
# The ToolsOutputParser can transform tool calls into Python objects (like Pydantic models)
for step in structured_llm.steps:
    print(type(step))

last_step = structured_llm.steps[1]  # This is the ToolsOutputParser object
print(last_step.pydantic_schemas)

# Manually parse a tool call for demonstration; typically the chain does this automatically
output_object = last_step._pydantic_parse(used_tool.tool_calls[0])
print(type(output_object))
print(output_object)


## Generalizing Structure Output to Tool Use
"""
Sometimes the model won't respond with the tool usage we want.
We can make multiple tools available to the model and bind them with bind_tools().
This allows more flexible usage of tools for tasks beyond structured output.
"""

task = "What is the address of the {college} at the University of Michigan?"
prompt = ChatPromptTemplate([("human", task)])
inputs = [{"college": "School of Information"}]

# Create a base LLM
llm_llama3_70b_instruct = ChatBedrockConverse(
    model="us.meta.llama3-1-70b-instruct-v1:0",
    region_name="us-east-1",
)

# Bind a tool (USPostalAddress) to the LLM
llama_address = llm_llama3_70b_instruct.bind_tools([USPostalAddress])


# We'll create a RunnableLambda to verify the LLM's response
def verify(msg):
    """
    Checks whether the LLM used any tools in its response.
    Raises an Exception if no tools were used.
    """
    print(f"Verifying message {msg}")
    if len(msg.tool_calls) <= 0:
        print("No tools were used in the chain, raising Exception")
        raise Exception("No tools were used in the chain")
    return msg


from langchain_aws.function_calling import ToolsOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables.base import RunnableLambda

# Build the initial chain
chain = prompt | llama_address | RunnableLambda(verify)

# Add retry logic (up to 15 attempts) in case the tool is not used at first
chain = chain.with_retry(stop_after_attempt=15, retry_if_exception_type=(Exception,))

# Use ToolsOutputParser to parse the final structured output
tools_parser = ToolsOutputParser(pydantic_schemas=[USPostalAddress])
chain = chain | tools_parser

# Finally, convert this chain to run over a list of inputs using .map()
chain = chain.map()

# Invoke the chain on our inputs
results = chain.invoke(inputs)
print("Final structured results:")
for item in results:
    print(item)

<class 'langchain_core.runnables.base.RunnableBinding'>
<class 'langchain_aws.function_calling.ToolsOutputParser'>
[<class '__main__.USPostalAddress'>]
<class '__main__.USPostalAddress'>
name='School of Information, University of Michigan' street='105 S State St' city='Ann Arbor' state=<USStateEnum.MI: 'MI'> zip='48109'


Verifying message content=[{'type': 'tool_use', 'name': 'USPostalAddress', 'input': {'name': 'School of Information, University of Michigan', 'street': '105 S State St', 'city': 'Ann Arbor', 'state': 'MI', 'zip': '48109'}, 'id': 'tooluse_EbN_wR9fTeysHBLk6SOwsQ'}] additional_kwargs={} response_metadata={'ResponseMetadata': {'RequestId': '8064be7e-52f6-45ea-a9e9-ccb2d972658f', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 04 Mar 2025 09:17:43 GMT', 'content-type': 'application/json', 'content-length': '393', 'connection': 'keep-alive', 'x-amzn-requestid': '8064be7e-52f6-45ea-a9e9-ccb2d972658f'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': 2057}} id='run-f0efda2c-a36a-4826-9098-6277b9b9e91e-0' tool_calls=[{'name': 'USPostalAddress', 'args': {'name': 'School of Information, University of Michigan', 'street': '105 S State St', 'city': 'Ann Arbor', 'state': 'MI', 'zip': '48109'}, 'id': 'tooluse_EbN_wR9fTeysHBLk6SOwsQ', 'type': 'tool_call'}] usage_metadata={